In [31]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')
import torch as tr 
import numpy as np 
import pandas as pd 
from collections import defaultdict
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from src.lda_model import LDASmoothed 
from src.generator import doc_generator 
from src.utils import (
    get_vocab_from_docs, 
    get_np_wct, 
    data_loader,
    text_pipeline, 
    process_documents,
) 
from src.text_pre_processor import (
    remove_accented_chars, 
    remove_special_characters, 
    remove_punctuation,
    remove_extra_whitespace_tabs,
    remove_stopwords,
)
from pprint import pprint 
import copy 

In [ ]:
doc_generator()

In [33]:
docs_raw_dict, raw_word_2_idx, raw_idx_2_word = data_loader('ap')

There are 2250 documents in the dataset
On average estimated document length is 406.66133333333335 words per document
There are 10473 unique vocab in the raw corpus


In [34]:
result = process_documents(docs_raw_dict, sample=True)

There are 300 documents in the dataset after processing
On average estimated document length is 10.0 words per document after processing
There are 2057 unique vocab in the corpus after processing


In [35]:
lda = LDASmoothed(
    docs = result['documents'],
    num_topics = 5, 
    word_ct_dict = result['vocab_doc_count_dict'], 
    word_ct_array = result['vocab_doc_count_array'],
)
#LDASmoothed(docs, 3, wct_dict)

Topic Dirichlet Prior, Alpha
torch.Size([5])

Word Dirichlet Prior, Eta
torch.Size([2057])

Var Inf - Word Dirichlet prior, Lambda
torch.Size([5, 2057])

Var Inf - Topic Dirichlet prior, Gamma
torch.Size([300, 5])

loop phi
looped
double
Var -Inf - Word wise Topic Multinomial/Categorical, Phi
torch.Size([300, 2057, 5])


In [37]:
{'name':'astrophysics',
                   'prob':[0.90, 0.025, 0.025, 0.025, 0.025],
                   'topic':'science',
                   },

({'name': 'astrophysics',
  'prob': [0.9, 0.025, 0.025, 0.025, 0.025],
  'topic': 'science'},)

In [36]:
lda.fit()

0->ELBO -27634.997578336068
Training started -> ELBO at init is :-27634.997578336068
Iteration 0, Delta Gamma = inf, the ELBO is -27634.997578336068
Iteration 1, Delta Gamma = 15856.303907289526, the ELBO is -24683.715353594853
Iteration 2, Delta Gamma = 2.894657750184471, the ELBO is -24679.619166531564
Iteration 3, Delta Gamma = 2.745779955966529, the ELBO is -24675.686550797946
Iteration 4, Delta Gamma = 2.7300636271863334, the ELBO is -24671.542362422897
Iteration 5, Delta Gamma = 2.823873730229845, the ELBO is -24666.90387943925
Iteration 6, Delta Gamma = 3.0345979913178502, the ELBO is -24661.45297503446


KeyboardInterrupt: 

In [9]:
lda._alpha_

tensor([1.1399, 1.0996, 1.0245, 1.0588, 1.1924, 1.0432, 1.0476, 1.0303, 1.1025,
        0.9284, 0.8418, 1.1272, 0.9701, 1.0061, 1.1151, 1.0796, 1.0086, 1.0041,
        1.0649, 1.1098], dtype=torch.float64)

In [10]:
lda._eta_

tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64)

In [11]:
lda._phi_

tensor([[[0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.0500],
         [0.0500, 0.0500, 0.0500,  ..., 0.0500, 0.0500, 0.